## Dependencies

In [1]:
import json, warnings, shutil
from jigsaw_utility_scripts import *
from transformers import TFXLMRobertaModel, XLMRobertaConfig
from tensorflow.keras.models import Model
from tensorflow.keras import optimizers, metrics, losses, layers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint


from transformers import TFAutoModel, AutoTokenizer

warnings.filterwarnings("ignore")

## TPU configuration

In [2]:
strategy, tpu = set_up_strategy()
print("REPLICAS: ", strategy.num_replicas_in_sync)
AUTO = tf.data.experimental.AUTOTUNE

Running on TPU  grpc://10.0.0.2:8470
REPLICAS:  8


# Load data

In [3]:
# database_base_path = '/kaggle/input/jigsaw-dataset-split-pb-roberta-large-192/'
# k_fold = pd.read_csv(database_base_path + '5-fold.csv')
# valid_df = pd.read_csv("/kaggle/input/jigsaw-multilingual-toxic-comment-classification/validation.csv", usecols=['comment_text', 'toxic', 'lang'])

# print('Train set samples: %d' % len(k_fold))
# print('Validation set samples: %d' % len(valid_df))
# display(k_fold.head())

# # Unzip files
# !tar -xvf /kaggle/input/jigsaw-dataset-split-pb-roberta-large-192/fold_1.tar.gz
# # !tar -xvf /kaggle/input/jigsaw-dataset-split-pb-roberta-large-192/fold_2.tar.gz
# # !tar -xvf /kaggle/input/jigsaw-dataset-split-pb-roberta-large-192/fold_3.tar.gz
# # !tar -xvf /kaggle/input/jigsaw-dataset-split-pb-roberta-large-192/fold_4.tar.gz
# # !tar -xvf /kaggle/input/jigsaw-dataset-split-pb-roberta-large-192/fold_5.tar.gz

# Model parameters

In [4]:
base_path = '/kaggle/input/jigsaw-transformers/XLM-RoBERTa/'

config = {
  "MAX_LEN": 192,
  "BATCH_SIZE": 16 * strategy.num_replicas_in_sync,
  "EPOCHS": 2,
  "LEARNING_RATE": 1e-5, 
  "ES_PATIENCE": 1,
  "N_FOLDS": 1,
  "base_model_path": base_path + 'tf-xlm-roberta-large-tf_model.h5',
  "config_path": base_path + 'xlm-roberta-large-config.json'
}

with open('config.json', 'w') as json_file:
    json.dump(json.loads(json.dumps(config)), json_file)

# Model

In [5]:
# module_config = XLMRobertaConfig.from_pretrained(config['config_path'], output_hidden_states=False)

# def model_fn(MAX_LEN):
#     input_ids = layers.Input(shape=(MAX_LEN,), dtype=tf.int32, name='input_ids')
    
#     base_model = TFXLMRobertaModel.from_pretrained(config['base_model_path'], config=module_config)
#     sequence_output = base_model({'input_ids': input_ids})
    
#     last_state = sequence_output[0]
#     cls_token = last_state[:, 0, :]
    
#     output = layers.Dense(1, activation='sigmoid', name='output')(cls_token)
    
#     model = Model(inputs=input_ids, outputs=output)
#     model.compile(optimizers.Adam(lr=config['LEARNING_RATE']), 
#                   loss=losses.BinaryCrossentropy(), 
#                   metrics=[metrics.BinaryAccuracy(), metrics.AUC()])
    
#     return model

# Train

In [6]:
# history_list = []

# for n_fold in range(config['N_FOLDS']):
#     tf.tpu.experimental.initialize_tpu_system(tpu)
#     print('\nFOLD: %d' % (n_fold+1))
#     # Load data
#     base_data_path = 'fold_%d/' % (n_fold+1)
#     x_train = np.load(base_data_path + 'x_train.npy')
#     y_train = np.load(base_data_path + 'y_train.npy')
#     x_valid = np.load(base_data_path + 'x_valid.npy')
#     x_valid_ml = np.load(database_base_path + 'x_valid.npy')
#     y_valid_ml = np.load(database_base_path + 'y_valid.npy')
    
#     step_size = x_train.shape[0] // config['BATCH_SIZE']

#     ### Delete data dir
#     shutil.rmtree(base_data_path)

#     # Train model
#     model_path = 'model_fold_%d.h5' % (n_fold+1)
#     es = EarlyStopping(monitor='val_loss', mode='min', patience=config['ES_PATIENCE'], 
#                        restore_best_weights=True, verbose=1)
#     checkpoint = ModelCheckpoint(model_path, monitor='val_loss', mode='min', 
#                                  save_best_only=True, save_weights_only=True, verbose=1)
    
#     with strategy.scope():
#         model = model_fn(config['MAX_LEN'])
#         history = model.fit(get_training_dataset(x_train, y_train, config['BATCH_SIZE'], AUTO),
#                         validation_data=(get_validation_dataset(x_valid_ml, y_valid_ml, config['BATCH_SIZE'], AUTO)),
#                         callbacks=[checkpoint, es],
#                         epochs=config['EPOCHS'], 
#                         steps_per_epoch=step_size,
#                         verbose=1).history
        
#     history_list.append(history)
    
#     # Make predictions
#     train_preds = model.predict(get_test_dataset(x_train, config['BATCH_SIZE'], AUTO))
#     valid_preds = model.predict(get_test_dataset(x_valid, config['BATCH_SIZE'], AUTO))
#     valid_ml_preds = model.predict(get_test_dataset(x_valid_ml, config['BATCH_SIZE'], AUTO))
    
#     k_fold.loc[k_fold['fold_%d' % (n_fold+1)] == 'train', 'pred_%d' % (n_fold+1)] = np.round(train_preds)
#     k_fold.loc[k_fold['fold_%d' % (n_fold+1)] == 'validation', 'pred_%d' % (n_fold+1)] = np.round(valid_preds)
#     valid_df['pred_%d' % (n_fold+1)] = np.round(valid_ml_preds)

## Model loss graph

In [7]:
# sns.set(style="whitegrid")
# for n_fold in range(config['N_FOLDS']):
#     print('Fold: %d' % (n_fold+1))
#     plot_metrics(history_list[n_fold])

# Model evaluation

In [8]:
# display(evaluate_model(k_fold, config['N_FOLDS']).style.applymap(color_map))

# Confusion matrix

In [9]:
# for n_fold in range(config['N_FOLDS']):
#     print('Fold: %d' % (n_fold+1))
#     train_set = k_fold[k_fold['fold_%d' % (n_fold+1)] == 'train']
#     validation_set = k_fold[k_fold['fold_%d' % (n_fold+1)] == 'validation'] 
#     plot_confusion_matrix(train_set['toxic'], train_set['pred_%d' % (n_fold+1)], 
#                           validation_set['toxic'], validation_set['pred_%d' % (n_fold+1)])

# Model evaluation by language

In [10]:
# display(evaluate_model_lang(valid_df, config['N_FOLDS']).style.applymap(color_map))

# Visualize predictions

In [11]:
# pd.set_option('max_colwidth', 120)
# display(k_fold[['comment_text', 'toxic'] + [c for c in k_fold.columns if c.startswith('pred')]].head(15))

In [12]:
def regular_encode(texts, tokenizer, maxlen=512):
    enc_di = tokenizer.batch_encode_plus(
        texts, 
        return_attention_masks=False, 
        return_token_type_ids=False,
        pad_to_max_length=True,
        max_length=maxlen
    )
    
    return np.array(enc_di['input_ids'])

def build_model(transformer, max_len=512):
    input_word_ids = layers.Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    sequence_output = transformer(input_word_ids)[0]
    cls_token = sequence_output[:, 0, :]
    out = layers.Dense(1, activation='sigmoid')(cls_token)
    
    model = Model(inputs=input_word_ids, outputs=out)
    model.compile(optimizers.Adam(lr=1e-5), loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [13]:
AUTO = tf.data.experimental.AUTOTUNE

# Configuration
EPOCHS = 2
BATCH_SIZE = 16 * strategy.num_replicas_in_sync
MAX_LEN = 192
MODEL = 'jplu/tf-xlm-roberta-large'

In [14]:
tokenizer = AutoTokenizer.from_pretrained(MODEL)

In [15]:
train1 = pd.read_csv("/kaggle/input/jigsaw-multilingual-toxic-comment-classification/jigsaw-toxic-comment-train.csv")
train2 = pd.read_csv("/kaggle/input/jigsaw-multilingual-toxic-comment-classification/jigsaw-unintended-bias-train.csv")
train2.toxic = train2.toxic.round().astype(int)

valid = pd.read_csv('/kaggle/input/jigsaw-multilingual-toxic-comment-classification/validation.csv')
test = pd.read_csv('/kaggle/input/jigsaw-multilingual-toxic-comment-classification/test.csv')
sub = pd.read_csv('/kaggle/input/jigsaw-multilingual-toxic-comment-classification/sample_submission.csv')

# Combine train1 with a subset of train2
train = pd.concat([
    train1[['comment_text', 'toxic']],
    train2[['comment_text', 'toxic']].query('toxic==1'),
    train2[['comment_text', 'toxic']].query('toxic==0').sample(n=100000, random_state=0)
])

In [16]:
x_train = regular_encode(train.comment_text.values, tokenizer, maxlen=MAX_LEN)
x_valid = regular_encode(valid.comment_text.values, tokenizer, maxlen=MAX_LEN)
x_test = regular_encode(test.content.values, tokenizer, maxlen=MAX_LEN)

y_train = train.toxic.values
y_valid = valid.toxic.values

In [17]:
train_dataset = (
    tf.data.Dataset
    .from_tensor_slices((x_train, y_train))
    .repeat()
    .shuffle(2048)
    .batch(BATCH_SIZE)
    .prefetch(AUTO)
)

valid_dataset = (
    tf.data.Dataset
    .from_tensor_slices((x_valid, y_valid))
    .batch(BATCH_SIZE)
    .cache()
    .prefetch(AUTO)
)

test_dataset = (
    tf.data.Dataset
    .from_tensor_slices(x_test)
    .batch(BATCH_SIZE)
)

In [18]:
module_config = XLMRobertaConfig.from_pretrained(config['config_path'], output_hidden_states=False)

with strategy.scope():
#     transformer_layer = TFAutoModel.from_pretrained(config['base_model_path'])
    transformer_layer = TFXLMRobertaModel.from_pretrained(config['base_model_path'], config=module_config)
    model = build_model(transformer_layer, max_len=MAX_LEN)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_word_ids (InputLayer)  [(None, 192)]             0         
_________________________________________________________________
tfxlm_roberta_model (TFXLMRo ((None, 192, 1024), (None 559890432 
_________________________________________________________________
tf_op_layer_strided_slice (T [(None, 1024)]            0         
_________________________________________________________________
dense (Dense)                (None, 1)                 1025      
Total params: 559,891,457
Trainable params: 559,891,457
Non-trainable params: 0
_________________________________________________________________


In [19]:
n_steps = x_train.shape[0] // BATCH_SIZE
train_history = model.fit(
    train_dataset,
    steps_per_epoch=n_steps,
    validation_data=valid_dataset,
    epochs=EPOCHS
)

Train for 3404 steps, validate for 63 steps
Epoch 1/2
3404/3404 [==============================] - 1825s 536ms/step - loss: 0.0679 - accuracy: 0.9737 - val_loss: 0.5837 - val_accuracy: 0.8469
Epoch 2/2
3404/3404 [==============================] - 1615s 475ms/step - loss: 0.0543 - accuracy: 0.9785 - val_loss: 0.3064 - val_accuracy: 0.8671


In [20]:
n_steps = x_valid.shape[0] // BATCH_SIZE
train_history_2 = model.fit(
    valid_dataset.repeat(),
    steps_per_epoch=n_steps,
    epochs=EPOCHS
)

Train for 62 steps
Epoch 1/2
62/62 [==============================] - 68s 1s/step - loss: 0.2344 - accuracy: 0.8943
Epoch 2/2
62/62 [==============================] - 140s 2s/step - loss: 0.1617 - accuracy: 0.9270


In [21]:
sub['toxic'] = model.predict(test_dataset, verbose=1)
sub.to_csv('submission.csv', index=False)

499/499 [==============================] - 110s 221ms/step
